## Cleans TRACE Corporate Bond Data

In [1]:
# Imports
import pandas as pd
import numpy as np

In [2]:
# Loads in subset of prof Yoshio's data
with pd.read_csv("prof_yoshio_data/trace_credit_spreads.csv", chunksize=1000) as reader:
    data = reader.get_chunk()

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 40 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   cusip_id                1000 non-null   object 
 1   date                    1000 non-null   int64  
 2   price                   1000 non-null   float64
 3   volume                  1000 non-null   int64  
 4   ISSUE_ID                1000 non-null   int64  
 5   ISSUER_ID               1000 non-null   int64  
 6   PROSPECTUS_ISSUER_NAME  1000 non-null   object 
 7   MATURITY                1000 non-null   int64  
 8   SECURITY_LEVEL          1000 non-null   object 
 9   OFFERING_AMT            1000 non-null   int64  
 10  OFFERING_DATE           1000 non-null   int64  
 11  OFFERING_PRICE          852 non-null    float64
 12  INTEREST_FREQUENCY      1000 non-null   int64  
 13  COUPON                  1000 non-null   float64
 14  standard                1000 non-null   i

In [174]:
# data.memory_usage(deep=True) \
# .gt(8000)

As expected all "object" (ie. string) columns use up the majority of the memory. The rest (float/int64s) are using 8 bytes, however these may also be reduced to 16 or 32.

In [13]:
def cols(col_names, data_type):
    '''
    Takes in list of column names and a datatype and generates a dictionary
    '''
    return {col_names[i]: data_type for i in range(len(col_names))}

In [19]:
def clean_data_types(df):
    '''
    Cleans datatypes for raw data
    '''
    df.columns = map(str.lower, df.columns)
    
    # String/text columns
    string_cols = ['cusip_id', 'cusip', 'prospectus_issuer_name']
    strings = cols(string_cols, 'string[pyarrow]')

    bool_cols = ['callable', 'make_whole', 'conv', 'junior']
    bools = cols(bool_cols, 'bool')

    category_cols = ['security_level', 'rating_spr', 'rating_mdy', 'standard', 'putop', 'industry_code']
    categories = cols(category_cols, 'category')
    try:
        df = df.drop(["sp_rat", "moody_rat", "rating", 'rating_date_spr', 'rating_date_mdy'], axis=1)
    except: pass
        
    # Float/Integer columns
    int8_cols = ['interest_frequency']
    int8s = cols(int8_cols, 'int8')

    df['sic_code'] = df['sic_code'].fillna(0)
    int16_cols = ['sic_code']
    int16s = cols(int16_cols, 'int16')

    int32_cols = ['volume', 'offering_amt', 'amount_outstanding', 'issue_id', 'issuer_id']
    int32s = cols(int32_cols, 'int32')

    float32_cols = ['price', 'offering_price', 'coupon', 'accrued_interest', 'tau',
                   'age', 'ytm', 'duration', 'tr_dirty_price', 'tr_ytm', 'cs']
    float32s = cols(float32_cols, 'float32')

    # Dates
    # formula to convert to datetime: df['date'].apply(lambda x: pd.to_datetime(str(x), format='%Y%m%d'))
    df['offering_date'] = df['offering_date'].fillna(0)
    date_cols = ['date', 'maturity', 'offering_date', 'month1']
    dates = cols(date_cols, 'int32')

    # Applies type changes
    types = strings | bools | categories | int8s | int16s | int32s | float32s | dates

    return df.astype(types)

In [192]:
df = clean_data_types(df)
# df.memory_usage(deep=True)

In [198]:
df.info()
# Memory usage decreased by about half

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 35 columns):
 #   Column                  Non-Null Count  Dtype   
---  ------                  --------------  -----   
 0   cusip_id                1000 non-null   string  
 1   date                    1000 non-null   int32   
 2   price                   1000 non-null   float32 
 3   volume                  1000 non-null   int32   
 4   issue_id                1000 non-null   int32   
 5   issuer_id               1000 non-null   int32   
 6   prospectus_issuer_name  1000 non-null   string  
 7   maturity                1000 non-null   int32   
 8   security_level          1000 non-null   category
 9   offering_amt            1000 non-null   int32   
 10  offering_date           1000 non-null   int32   
 11  offering_price          852 non-null    float32 
 12  interest_frequency      1000 non-null   int8    
 13  coupon                  1000 non-null   float32 
 14  standard                1

In [204]:
# Loads in whole data
data = pd.read_csv("prof_yoshio_data/trace_credit_spreads.csv")

C:\Users\sroma\AppData\Local\Temp\ipykernel_25248\3165405631.py:2: DtypeWarning: Columns (19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("prof_yoshio_data/trace_credit_spreads.csv")


14090424.827 KB


In [221]:
# Cleans data
cleaned_data = clean_data_types(data)

14090424.827 KB


In [ ]:
# Saves cleaned data
cleaned_data.to_csv('cleaned_data.csv')

In [222]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18833993 entries, 0 to 18833992
Data columns (total 40 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   cusip_id                object 
 1   date                    int64  
 2   price                   float64
 3   volume                  float64
 4   issue_id                int64  
 5   issuer_id               int64  
 6   prospectus_issuer_name  object 
 7   maturity                int64  
 8   security_level          object 
 9   offering_amt            float64
 10  offering_date           float64
 11  offering_price          float64
 12  interest_frequency      int64  
 13  coupon                  float64
 14  standard                int64  
 15  putop                   int64  
 16  conv                    int64  
 17  junior                  int64  
 18  cusip                   object 
 19  callable                object 
 20  make_whole              object 
 21  industry_code           int64

In [224]:
cleaned_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18833993 entries, 0 to 18833992
Data columns (total 35 columns):
 #   Column                  Dtype   
---  ------                  -----   
 0   cusip_id                string  
 1   date                    int32   
 2   price                   float32 
 3   volume                  int32   
 4   issue_id                int32   
 5   issuer_id               int32   
 6   prospectus_issuer_name  string  
 7   maturity                int32   
 8   security_level          category
 9   offering_amt            int32   
 10  offering_date           int32   
 11  offering_price          float32 
 12  interest_frequency      int8    
 13  coupon                  float32 
 14  standard                category
 15  putop                   category
 16  conv                    bool    
 17  junior                  bool    
 18  cusip                   string  
 19  callable                bool    
 20  make_whole              bool    
 21  indust

# Aggregates Data to Monthly

In [2]:
# Imports
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
from dateutil.relativedelta import *
from pandas.tseries.offsets import *

In [132]:
# Loads in subset of cleaned data
with pd.read_csv("cleaned_data.csv", chunksize=100_000, index_col=0) as reader:
    data = reader.get_chunk()

# csv doesn't preserve data types
data = clean_data_types(data)

In [133]:
print(f''' With the subset of data there are: 
    - {len(data['issuer_id'].unique())} unique issuers 
    - {len(data['issue_id'].unique())} unique bonds  
    - {len(data['date'].sort_values().unique())} days in the sample
    ''')

 With the subset of data there are: 
    - 43 unique issuers 
    - 192 unique bonds  
    - 4913 days in the sample
    


## Sample Exclusion Criteria
Based on Bai, Bali & Wen (2019):

1. Remove bonds that are not listed or traded in the US public market, which include bonds issued through private placement, bonds issued under the 144A rule, bonds that do not trade in US dollars, and bond issuers not in the jurisdiction of the United States.
2. Remove bonds that are structured notes, mortgage backed or asset backed, agency backed, or equity linked.
3. Remove convertible bonds since this option feature distorts the return calculation and makes it impossible to compare the returns of convertible and nonconvertible bonds.
4. Remove bonds that trade under 5 or above 1000. 
5. Remove bonds that have a floating coupon rate, which means the sample comprises only bonds with a fixed or zero coupon. This rule is applied based on the consideration of the accuracy in bond return calculation, given the challenge in tracking a floating-coupon bond’s cash flows.
6. Remove bonds that have less than one year to maturity. This rule is applied to all major corporate bond indices such as the Barclays Capital Corporate Bond Index, the Bank of America Merrill Lynch Corporate Master Index, and the Citi Fixed Income Indices. If a bond has less than one year to maturity, it will be delisted from major bond indices; hence, index-tracking investors will change their holding positions. This operation will distort the return calculation for bonds with less than one year to maturity; thus, we remove them from our sample.
7. For intraday data, we also eliminate bond transactions that are labeled as when-issued, locked-in, or have special sales conditions, and that have more than a twoday settlement.
8. Remove transaction records that are canceled and adjust records that are subsequently corrected or reversed.
9. Remove transaction records that have trading volume less than $10,000

### 3. Convertible Bonds

In [156]:
def filter_bond_convertible(data):
    '''
    Function which filters out convertible bonds.
    '''
    
    return data[data['conv'] == False]

In [157]:
data = filter_bond_convertible(data)

### 4. Bonds with irregular trading prices

In [162]:
def filter_bond_prices(data):
    '''
    Function which filters out bonds that trade below $5 or above $1000.
    '''
    
    return data[(data['price'].gt(5)) | (data['price'].lt(1000))]

In [163]:
data = filter_bond_prices(data)

### 6. Bonds with less than 1-year to maturity
Perhaps worthwhile to keep seperate as a robustness check?

In [134]:
def filter_bond_maturity(data):
    '''
    Function which filters out bonds with less than 1-year to maturity.
    Saves bonds with less than 1-year to maturity in a seperate file.
    '''
    data['date'] = pd.to_datetime(data['date'], format='%Y%m%d')
    data['maturity'] = pd.to_datetime(data['maturity'], format='%Y%m%d')

    data = data.sort_values(by='date')
    data['days_to_maturity'] = data['maturity'] - data['date']

    # Filters out bonds with less than 1-year to maturity
    data_one_year_bonds = data[data['days_to_maturity'].le('365 days')]
    data_one_year_bonds.to_csv('bonds_1_year_maturity.csv')
    
    return data[data['days_to_maturity'].gt('365 days')]

In [135]:
data = filter_bond_maturity(data)

In [137]:
data.shape

(96721, 36)

### 9. Bonds with small trading volumes

In [143]:
def filter_bond_volume(data):
    '''
    Function which filters daily volumes of less than $10,000.
    '''
    
    return data[data['volume'].gt(10000)]

In [144]:
data = filter_bond_volume(data)

In [146]:
data.shape

(95529, 36)

### Monthly Return Aggregation

The monthly corporate bond return at time $t$ is computed as 

<font size=6>
$
r_{i,t} = \frac{P_{i,t} + AI_{i,t} + C_{i,t}}{P_{i,t−1} + AI_{i,t−1}} − 1
$
</font>

Where $P_{i,t}$ is month $t$'s bond clean price, $AI_{i,t}$ is the accrued-interest, and $C_{i,t}$ is the coupon.

Excess returns are computed by deducting the 1-month Treasury bill rate.

Further, following Bai, Bali & Wen (2019), returns can be computed as either end of month t relative to end of month t-1, or end of month t to begining of month t. Where the end/beginning of the month is defined with a 5 day window. The date which is closest to the last/firsy trading day is used. If both such return calculations are available, then the end-to-end is prefered.

#### End-to-End Method

In [396]:
data.shape

(95529, 36)

In [412]:
def end_month_returns(data):
    '''
    Function to compute monthly returns using the end-to-end method.
    '''
    # Ensures that data is sorted in chronological order
    data = data.sort_values(by='date')
    
    # Gets 5 days from the end of the month
    data['month_end'] = data['date'] + MonthEnd(0) - pd.Timedelta(days=5)
    
    # Groups by issue id and year-month: returns the last observation
    data = data.groupby(['issue_id', data['date'].dt.year, data['date'].dt.month]).tail(1).copy()
    
    # Creates the P + AI + C for time t
    data['full_price_t'] = data['price'] + data['accrued_interest'] + data['coupon']
    data['price_t'] = data['price'] + data['accrued_interest']
    
    # Lags P + AI
    data['price_t1'] = data.groupby('issue_id')['price_t'].shift(1)
    
    # Computes the return only if the transaction date is within 5 days of end of month
    data['return'] = np.where(~(data['date'] < data['month_end']), 
                               (data['full_price_t'] / data['price_t1']) - 1, np.nan)
    
    # Price return used for validation
    data['price_return'] = np.where(~(data['date'] < data['month_end']), 
                               (data['price'] / data.groupby('issue_id')['price'].shift(1)) - 1, np.nan)
    
    # Drops observations where there is no return, assigns month end value
    data = data.dropna(subset='return')
    data['date'] = data['date'] + MonthEnd(0)

    return data 

In [413]:
end_data = end_month_returns(data)

In [414]:
end_data.shape

(6702, 42)

#### Start-to-End Method

In [548]:
data.shape

(95529, 36)

In [561]:
def start_month_returns(data):
    '''
    Function to compute monthly returns using the start-to-end method.
    '''
    # Ensures that data is sorted in chronological order
    data = data.sort_values(by='date')
    
    # 5 days from start and end of month
    data['month_end'] = data['date'] + MonthEnd(0) - pd.Timedelta(days=5)
    data['month_start'] = data['date'] + MonthEnd(0) - MonthBegin(1) + pd.Timedelta(days=5)
    data['date_t'] = data['date'] + MonthEnd(0)
    
    # Seperate first and last trade dataframes
    first = data.groupby(['issue_id', data['date'].dt.year, data['date'].dt.month]).head(1).copy()
    last = data.groupby(['issue_id', data['date'].dt.year, data['date'].dt.month]).tail(1).copy()
    
    # Respective price only if date requirement is satisifed 
    first['price_t'] = np.where((first['date'] < first['month_start']),
                                 first['price'] + first['accrued_interest'], np.nan)
    last['full_price_t'] = np.where(~(last['date'] < last['month_end']),
                                      last['price'] + last['accrued_interest'] + last['coupon'], np.nan)
    
    # Merges first and last on true month end
    merged = pd.merge(first, last[['issue_id', 'date_t', 'full_price_t']], on=['issue_id', 'date_t'])

    # Computes the return
    merged['return'] = (merged['full_price_t'] / merged['price_t']) - 1
    
    # Drops observations where there is no return, assigns month end value
    data = merged.dropna(subset='return').copy()
    data['date'] = data['date'] + MonthEnd(0)
    
    return data

In [562]:
start_data = start_month_returns(data)

In [563]:
start_data.shape

(5516, 42)

In [554]:
start_data

,cusip_id,date,price,volume,issue_id,issuer_id,prospectus_issuer_name,maturity,security_level,offering_amt,...,tr_dirty_price,tr_ytm,cs,days_to_maturity,month_end,month_start,date_t,price_t,full_price_t,return
0,00077QAA8,2003-12-31,107.513184,9865000,103438,6097,ABN AMRO BK N V CHICAGO BRH,2005-05-31,SENS,1000000,...,108.063522,1.781719,0.343266,547 days,2003-12-26,2003-12-06,2003-12-31,107.533104,114.871445,0.068243
1,00130HAV7,2004-11-30,110.738655,3925000,123511,11,AES CORP,2008-06-15,SEN,400000,...,122.843643,3.018593,2.421694,1322 days,2004-11-25,2004-11-06,2004-11-30,114.061745,123.150726,0.079685
2,00130HAU9,2005-02-28,112.541664,9000000,116400,11,AES CORP,2011-02-15,SEN,600000,...,131.318115,3.789522,2.552284,2205 days,2005-02-23,2005-02-06,2005-02-28,116.641525,123.068718,0.055102
3,00130HAV7,2005-04-30,105.181816,110000,123511,11,AES CORP,2008-06-15,SEN,400000,...,116.915131,3.939757,2.973959,1171 days,2005-04-25,2005-04-06,2005-04-30,107.753929,116.995193,0.085763
4,001546AG5,2005-07-31,85.255745,2175000,155823,13,AK STL CORP,2012-06-15,SEN,550000,...,123.438400,3.922578,6.896482,2541 days,2005-07-26,2005-07-06,2005-07-31,85.594543,101.883713,0.190306
5,00105DAB1,2007-01-31,106.500000,120000,201071,37707,AES GENER S A,2014-03-25,SEN,399645,...,119.176834,4.679144,1.682805,2639 days,2007-01-26,2007-01-06,2007-01-31,108.551102,116.211929,0.070573
6,00130HBC8,2007-02-28,105.516357,10700000,180991,11,AES CORP,2014-03-01,SEN,500000,...,120.562111,4.837404,1.918227,2585 days,2007-02-23,2007-02-06,2007-02-28,108.791908,116.664490,0.072364
7,00105DAB1,2007-12-31,105.387840,510000,201071,37707,AES GENER S A,2014-03-25,SEN,399645,...,123.662514,3.535811,2.908904,2304 days,2007-12-26,2007-12-06,2007-12-31,106.809540,115.471161,0.081094
8,001055AD4,2011-04-30,106.483368,270000,510620,33631,AFLAC INC,2039-12-17,SEN,400000,...,142.371185,4.406859,1.995740,10487 days,2011-04-25,2011-04-06,2011-04-30,108.473755,117.567894,0.083837
9,00164VAC7,2013-02-28,100.010231,7425000,585617,42397,AMC NETWORKS INC,2022-12-15,SEN,600000,...,125.842178,1.931944,2.816068,3604 days,2013-02-23,2013-02-06,2013-02-28,100.636604,105.592705,0.049248


In [524]:
test = data[(data['issue_id'] == 55723) | (data['issue_id'] == 76070)].copy()
test['month_end'] = test['date'] + MonthEnd(0) - pd.Timedelta(days=5)
test['month_start'] = test['date'] + MonthEnd(0) - MonthBegin(1) + pd.Timedelta(days=5)
test['date_t'] = test['date'] + MonthEnd(0)

In [525]:
first = test.groupby([test['date'].dt.year, test['date'].dt.month]).head(1).copy()
last = test.groupby([test['date'].dt.year, test['date'].dt.month]).tail(1).copy()

In [526]:
last['full_price_t'] = np.where(~(last['date'] < last['month_end']), 
                                  last['price'] + last['accrued_interest'] + last['coupon'], np.nan)

first['price_t'] = np.where((first['date'] < first['month_start']), 
                                  first['price'] + first['accrued_interest'], np.nan)

In [535]:
merged = pd.merge(first, last[['issue_id', 'date_t', 'full_price_t']], on=['issue_id', 'date_t'])

In [536]:
merged['return'] = (merged['full_price_t'] / merged['price_t']) - 1

In [537]:
merged.head()

,cusip_id,date,price,volume,issue_id,issuer_id,prospectus_issuer_name,maturity,security_level,offering_amt,...,tr_dirty_price,tr_ytm,cs,days_to_maturity,month_end,month_start,date_t,price_t,full_price_t,return
0,00077QAG5,2002-07-01,109.190002,4000000,55723,6097,ABN AMRO BK N V CHICAGO BRH,2007-06-18,SENS,750000,...,113.762276,4.089739,0.920484,1813 days,2002-07-26,2002-07-06,2002-07-31,109.443077,119.118088,0.088402
1,00077QAG5,2002-08-01,105.827003,60000,55723,6097,ABN AMRO BK N V CHICAGO BRH,2007-06-18,SENS,750000,...,117.340263,3.427217,2.309816,1782 days,2002-08-26,2002-08-06,2002-08-31,106.683563,119.546104,0.120567
2,00077QAG5,2002-09-03,109.540001,50000,55723,6097,ABN AMRO BK N V CHICAGO BRH,2007-06-18,SENS,750000,...,119.898201,2.975083,1.891342,1749 days,2002-09-25,2002-09-06,2002-09-30,111.038979,123.075592,0.108400
3,00077QAG5,2002-10-02,112.563156,16450000,55723,6097,ABN AMRO BK N V CHICAGO BRH,2007-06-18,SENS,750000,...,121.612839,2.679003,1.479867,1720 days,2002-10-26,2002-10-06,2002-10-31,114.626678,121.378075,0.058899
4,00077QAG5,2002-11-04,110.627296,675000,55723,6097,ABN AMRO BK N V CHICAGO BRH,2007-06-18,SENS,750000,...,120.992126,2.871157,1.676800,1687 days,2002-11-25,2002-11-06,2002-11-30,113.333237,122.315201,0.079253
